In [116]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import pandas as pd
from pandas_datareader import data 
import matplotlib.pyplot as plt

sns.set(style='darkgrid', context='talk', palette='Dark2')

Example of web scrapping on Tesla

In [117]:
start_date = '2000-01-01'
end_date = '2022-10-31'
microsoft = data.DataReader('MSFT', 'yahoo', start_date, end_date)
microsoft

High         Low        Open       Close      Volume  \
Date                                                                     
2000-01-03   59.312500   56.000000   58.687500   58.281250  53228400.0   
2000-01-04   58.562500   56.125000   56.781250   56.312500  54119000.0   
2000-01-05   58.187500   54.687500   55.562500   56.906250  64059600.0   
2000-01-06   56.937500   54.187500   56.093750   55.000000  54976600.0   
2000-01-07   56.125000   53.656250   54.312500   55.718750  62013600.0   
...                ...         ...         ...         ...         ...   
2022-10-25  251.039993  245.830002  247.259995  250.660004  34775500.0   
2022-10-26  238.300003  230.059998  231.169998  231.320007  82543200.0   
2022-10-27  233.690002  225.779999  231.039993  226.750000  40424600.0   
2022-10-28  236.600006  226.050003  226.240005  235.869995  40647700.0   
2022-10-31  234.919998  231.149994  233.759995  232.130005  28357300.0   

             Adj Close  
Date                    
2000-01-03   36.555382  
2000-01-04   35.320545  
2000-01-05   35.692932  
2000-01-06   34.497299  
2000-01-07   34.948109  
...                ...  
2022-10-25  250.660004  
2022-10-26  231.320007  
2022-10-27  226.750000  
2022-10-28  235.869995  
2022-10-31  232.130005  

[5745 rows x 6 columns]

Having imported the appropriate tools, getting market data from a free online source, such as Yahoo Finance, is super easy. Since pandas has a simple remote data access for the Yahoo Finance API data, this is as simple as:

In [118]:
def web_scrapper():
    tickers = pd.read_csv("CSV/constituents.csv")
    tickers_symbol = tickers['Symbol']

    # Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
    # tickers = ['AAPL', 'MSFT', 'GOOG', 'META']

    # We would like all available data from 01/01/2000 until 2022-10-31.
    start_date = '2000-01-01'
    end_date = '2022-10-31'

    # User pandas_reader.data.DataReader to load the desired data.
    df = data.DataReader(tickers_symbol, 'yahoo', start_date, end_date) # I commented this line because I saved the scrapped data into a .csv
    microsoft = data.DataReader('MSFT', 'yahoo', start_date, end_date)
    close = df['Close']
    volume = df['Volume']

    #Let us assume we are interested in working with the Close prices which have been already been adjusted by Yahoo finance to account for stock splits. We want to make sure that all weekdays are included in our dataset, which is very often desirable for quantitative trading strategies.

    #Of course, some of the weekdays might be public holidays in which case no price will be available. For this reason, we will fill the missing prices with the latest available prices:

    all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')  # B stands for business day frequency
    close = close.reindex(all_weekdays)
    close = close.fillna(method='ffill')
    close.set_index(microsoft.index)    
    volume = volume.reindex(all_weekdays)
    volume = volume.fillna(method='ffill')
    volume.set_index(microsoft.index)   
    df = df.reindex(all_weekdays)
    df = df.fillna(method='ffill')
    df.set_index(microsoft.index)   

    # df.to_csv("CSV/sp500.csv",index=False)
    # close.to_csv("CSV/sp500-close.csv",index=False)
    # volume.to_csv("CSV/sp500-volume.csv", index=False)

    df.to_csv("CSV/sp5002.csv",index=False)
    close.to_csv("CSV/sp5002-close.csv",index=False)
    volume.to_csv("CSV/sp5002-volume.csv", index=False)

    return df, close, volume


In [119]:
# web_scrapper()

In [120]:
sp500 = pd.read_csv("CSV/sp500.csv")
close = pd.read_csv("CSV/sp500-close.csv")
volume = pd.read_csv("CSV/sp500-volume.csv")

/home/joaodocarglass/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,2

#### Preparing the Data

Let us assume we are interested in working with the Close prices which have been already been adjusted by Yahoo finance to account for stock splits. We want to make sure that all weekdays are included in our dataset, which is very often desirable for quantitative trading strategies.

Of course, some of the weekdays might be public holidays in which case no price will be available. For this reason, we will fill the missing prices with the latest available prices:

Let's build a function to drop columns with nulls percentage above 25

In [121]:
def drop_null_columns(data):
    nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    # nulls_percent_df.head()
    columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0]
    # print(len(columns_above_threshold['column_name']))
    drop_columns_list = list(columns_above_threshold['column_name'])
    # print(drop_columns_list)
    data = data.drop(columns=drop_columns_list)
    data.shape
    return data

In [122]:
close.isna().sum()

MMM        0
AOS        0
ABT        0
ABBV    3392
ABMD       0
        ... 
KSU     5956
PBCT    5956
VIAC    5956
WLTW    5956
XLNX    5956
Length: 505, dtype: int64

In [123]:
close = drop_null_columns(close)

In [124]:
volume = drop_null_columns(volume)

In [125]:
close.isna().sum()

MMM     0
AOS     0
ABT     0
ABMD    0
ATVI    0
       ..
WMB     0
XEL     0
YUM     0
ZBRA    0
ZION    0
Length: 364, dtype: int64

#### Now we are going to create 100 random portfolios for the close and the volume
#### Each sample will be of 5 samples from the whole population

In [126]:
close

MMM        AOS        ABT        ABMD       ATVI        ADM  \
0      47.187500   3.614583  15.711531   18.250000   1.369792  10.884354   
1      45.312500   3.562500  15.262630   17.812500   1.328125  10.770975   
2      46.625000   3.552083  15.234574   18.000000   1.333333  10.600907   
3      50.375000   3.500000  15.767643   18.031250   1.307292  10.657596   
4      51.375000   3.645833  15.935981   17.937500   1.343750  10.827664   
...          ...        ...        ...         ...        ...        ...   
5951  118.500000  52.099998  97.949997  264.290009  72.500000  91.139999   
5952  122.699997  51.950001  98.419998  268.250000  72.480003  93.589996   
5953  122.800003  51.900002  96.930000  267.579987  72.500000  94.129997   
5954  126.599998  54.380001  99.489998  258.059998  72.849998  94.879997   
5955  125.790001  54.779999  98.940002  252.080002  72.800003  96.980003   

            ADBE        AMD        AES        AFL  ...       WELL         WST  \
0      16.390625  15.500000  36.250000  11.187500  ...  15.375000    7.562500   
1      15.015625  14.625000  34.812500  10.890625  ...  15.312500    7.640625   
2      15.312500  15.000000  35.187500  10.828125  ...  15.750000    7.609375   
3      15.437500  16.000000  35.468750  11.046875  ...  16.562500    7.531250   
4      16.187500  16.250000  36.156250  11.265625  ...  16.812500    7.609375   
...          ...        ...        ...        ...  ...        ...         ...   
5951  323.790009  61.470001  25.190001  61.950001  ...  60.360001  249.210007   
5952  320.480011  59.730000  25.450001  61.959999  ...  60.400002  254.399994   
5953  318.649994  58.599998  25.770000  62.700001  ...  59.939999  221.229996   
5954  325.679993  62.009998  26.459999  64.790001  ...  61.500000  225.509995   
5955  318.500000  60.060001  26.160000  65.110001  ...  61.040001  230.100006   

            WDC         WY         WHR        WMB        XEL         YUM  \
0      3.937500  69.875000   61.812500  23.198961  19.000000    6.706057   
1      4.687500  67.250000   59.437500  22.684469  19.437500    6.571262   
2      4.375000  70.812500   60.312500  24.087631  20.187500    6.604960   
3      4.562500  74.125000   60.812500  24.602125  20.000000    6.548796   
4      5.625000  71.687500   61.000000  25.256933  20.000000    6.402768   
...         ...        ...         ...        ...        ...         ...   
5951  35.290001  30.910000  137.000000  31.850000  62.400002  111.239998   
5952  35.279999  30.959999  134.740005  32.150002  62.570000  112.269997   
5953  34.340000  31.010000  135.029999  32.500000  63.250000  113.910004   
5954  35.500000  31.100000  139.910004  32.669998  65.370003  117.879997   
5955  34.369999  30.930000  138.240005  32.730000  65.110001  118.250000   

            ZBRA       ZION  
0      25.027779  55.500000  
1      24.666668  52.812500  
2      25.138889  52.750000  
3      23.777779  53.500000  
4      23.513889  53.625000  
...          ...        ...  
5951  274.579987  51.509998  
5952  273.179993  50.400002  
5953  275.290009  50.380001  
5954  288.000000  51.540001  
5955  283.220001  51.939999  

[5956 rows x 364 columns]

In [127]:
all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')  # B stands for business day frequency

microsoft = microsoft.reindex(all_weekdays)
microsoft = microsoft.fillna(method='ffill')

In [128]:
microsoft.index

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14',
               ...
               '2022-10-18', '2022-10-19', '2022-10-20', '2022-10-21',
               '2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27',
               '2022-10-28', '2022-10-31'],
              dtype='datetime64[ns]', length=5956, freq='B')

In [129]:
close.set_index(microsoft.index)

MMM        AOS        ABT        ABMD       ATVI  \
2000-01-03   47.187500   3.614583  15.711531   18.250000   1.369792   
2000-01-04   45.312500   3.562500  15.262630   17.812500   1.328125   
2000-01-05   46.625000   3.552083  15.234574   18.000000   1.333333   
2000-01-06   50.375000   3.500000  15.767643   18.031250   1.307292   
2000-01-07   51.375000   3.645833  15.935981   17.937500   1.343750   
...                ...        ...        ...         ...        ...   
2022-10-25  118.500000  52.099998  97.949997  264.290009  72.500000   
2022-10-26  122.699997  51.950001  98.419998  268.250000  72.480003   
2022-10-27  122.800003  51.900002  96.930000  267.579987  72.500000   
2022-10-28  126.599998  54.380001  99.489998  258.059998  72.849998   
2022-10-31  125.790001  54.779999  98.940002  252.080002  72.800003   

                  ADM        ADBE        AMD        AES        AFL  ...  \
2000-01-03  10.884354   16.390625  15.500000  36.250000  11.187500  ...   
2000-01-04  10.770975   15.015625  14.625000  34.812500  10.890625  ...   
2000-01-05  10.600907   15.312500  15.000000  35.187500  10.828125  ...   
2000-01-06  10.657596   15.437500  16.000000  35.468750  11.046875  ...   
2000-01-07  10.827664   16.187500  16.250000  36.156250  11.265625  ...   
...               ...         ...        ...        ...        ...  ...   
2022-10-25  91.139999  323.790009  61.470001  25.190001  61.950001  ...   
2022-10-26  93.589996  320.480011  59.730000  25.450001  61.959999  ...   
2022-10-27  94.129997  318.649994  58.599998  25.770000  62.700001  ...   
2022-10-28  94.879997  325.679993  62.009998  26.459999  64.790001  ...   
2022-10-31  96.980003  318.500000  60.060001  26.160000  65.110001  ...   

                 WELL         WST        WDC         WY         WHR  \
2000-01-03  15.375000    7.562500   3.937500  69.875000   61.812500   
2000-01-04  15.312500    7.640625   4.687500  67.250000   59.437500   
2000-01-05  15.750000    7.609375   4.375000  70.812500   60.312500   
2000-01-06  16.562500    7.531250   4.562500  74.125000   60.812500   
2000-01-07  16.812500    7.609375   5.625000  71.687500   61.000000   
...               ...         ...        ...        ...         ...   
2022-10-25  60.360001  249.210007  35.290001  30.910000  137.000000   
2022-10-26  60.400002  254.399994  35.279999  30.959999  134.740005   
2022-10-27  59.939999  221.229996  34.340000  31.010000  135.029999   
2022-10-28  61.500000  225.509995  35.500000  31.100000  139.910004   
2022-10-31  61.040001  230.100006  34.369999  30.930000  138.240005   

                  WMB        XEL         YUM        ZBRA       ZION  
2000-01-03  23.198961  19.000000    6.706057   25.027779  55.500000  
2000-01-04  22.684469  19.437500    6.571262   24.666668  52.812500  
2000-01-05  24.087631  20.187500    6.604960   25.138889  52.750000  
2000-01-06  24.602125  20.000000    6.548796   23.777779  53.500000  
2000-01-07  25.256933  20.000000    6.402768   23.513889  53.625000  
...               ...        ...         ...         ...        ...  
2022-10-25  31.850000  62.400002  111.239998  274.579987  51.509998  
2022-10-26  32.150002  62.570000  112.269997  273.179993  50.400002  
2022-10-27  32.500000  63.250000  113.910004  275.290009  50.380001  
2022-10-28  32.669998  65.370003  117.879997  288.000000  51.540001  
2022-10-31  32.730000  65.110001  118.250000  283.220001  51.939999  

[5956 rows x 364 columns]

In [130]:
def random_portfolios(data):
    import random
    list_of_portfolios = []
    tickers_data = list(data.columns.values)
    for i in range(100):
        portofolio_aleatorio = random.sample(tickers_data, 5)
        list_of_portfolios.append(data[portofolio_aleatorio])
    return list_of_portfolios

In [131]:
list_of_portfolios_close = random_portfolios(close)

In [132]:
list_of_portfolios_volume = random_portfolios(volume)

In [133]:
list_of_portfolios_close[0]

ESS         RSG        CHD          F         PPG
0      33.312500    9.416667   4.333333  28.782394   30.000000
1      32.937500    8.833333   4.229167  27.820692   28.968750
2      32.812500    8.666667   4.229167  27.923731   30.062500
3      33.125000    8.625000   4.375000  27.958078   31.500000
4      34.250000    9.083333   4.541667  30.018869   32.062500
...          ...         ...        ...        ...         ...
5951  236.449997  135.130005  75.620003  12.830000  112.959999
5952  228.259995  133.619995  76.559998  12.820000  112.820000
5953  221.570007  134.639999  76.510002  13.000000  112.360001
5954  222.240005  133.179993  76.370003  13.260000  114.160004
5955  222.240005  132.619995  74.129997  13.370000  114.180000

[5956 rows x 5 columns]

In [160]:
best_portfolio = [0]
for x in range(100):
    stock_performance = []
    for i in list_of_portfolios_close[x]:
        stock_performance.append(list_of_portfolios_close[x][i][-1:].values[0] - list_of_portfolios_close[x][i][0:].values[0])
    
    if np.mean(best_portfolio) >= np.mean(stock_performance):
        best_portfolio = stock_performance
best_portfolio

[-13.9399995803833,
 81.88999938964844,
 -8.601249694824219,
 21.887500762939453,
 -1328.8333740234375]

In [150]:
np.mean(stock_performance)

45.29437341690063

In [136]:
list_of_portfolios_close[1]

AVY       NLOK           C         PXD         CVX
0      71.375000   7.109375  397.500000    8.437500   41.812500
1      70.437500   6.781250  373.125000    8.562500   41.812500
2      72.750000   6.375000  388.125000    8.312500   42.562500
3      75.312500   6.007813  406.875000    8.500000   44.375000
4      75.562500   6.484375  405.000000    8.750000   45.156250
...          ...        ...         ...         ...         ...
5951  177.009995  22.250000   45.320000  264.829987  174.929993
5952  163.240005  22.180000   45.410000  265.489990  177.089996
5953  168.429993  21.830000   45.439999  265.839996  177.899994
5954  171.630005  22.559999   46.130001  257.309998  179.979996
5955  169.550003  22.530001   45.860001  256.410004  180.899994

[5956 rows x 5 columns]

In [137]:
stock_performance = []
for i in list_of_portfolios_close[1]:
    stock_performance.append(list_of_portfolios_close[1][i][-1:].values[0] - list_of_portfolios_close[1][i][0:].values[0])
stock_performance

[98.17500305175781,
 15.420625686645508,
 -351.63999938964844,
 247.97250366210938,
 139.0874938964844]

In [138]:
list_of_portfolios_volume[0]

AFL         EBAY        MOS        PGR        MTCH
0     2388800.0  116192102.0   816800.0  4741200.0   6276869.0
1     2716400.0   80316403.0   484600.0  7713600.0   4952001.0
2     1354800.0  104891846.0   703600.0  5454000.0   7600292.0
3     2027200.0  104893747.0   871300.0  3638400.0  10764372.0
4     2262000.0   51260774.0   890200.0  4105200.0   4478111.0
...         ...          ...        ...        ...         ...
5951  2461900.0    6544200.0  3206100.0  2473200.0   3060100.0
5952  2276300.0    5856200.0  3820700.0  1893100.0   3319100.0
5953  2345800.0    6130400.0  3115700.0  2173600.0   3855000.0
5954  2583400.0    7747000.0  2510700.0  2483400.0   8201800.0
5955  5193600.0    6860700.0  4023900.0  2062900.0   5022800.0

[5956 rows x 5 columns]

In [139]:
list_of_portfolios[0]['GLW'][-1:].values[0]

KeyError: 'GLW'

In [ ]:
stock_performance = list_of_portfolios[0]['GLW'][-1:].values[0] - list_of_portfolios[0]['GLW'][0:].values[0]

In [ ]:
stock_performance

In [ ]:
list(close.columns.values)[0]

In [ ]:
def close_print(stock_symbol):
    
    df = data.DataReader(stock_symbol, 'yahoo', start_date, end_date)
    close = df['Close']
    all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')  # B stands for business day frequency
    close = close.reindex(all_weekdays)
    close = close.fillna(method='ffill')

    for i in stock_symbol:
        # Get the ticker timeseries. This now returns a Pandas Series object indexed by date.
        ticker = close.loc[:, i]

        # Calculate the 20 and 100 days moving averages of the closing prices
        short_rolling_ticker = ticker.rolling(window=20).mean()
        long_rolling_ticker = ticker.rolling(window=100).mean()

        # Plot everything by leveraging the very powerful matplotlib package
        fig, ax = plt.subplots(figsize=(18,11))

        plt.title(i)

        ax.plot(ticker.index, ticker, label=i)
        ax.plot(short_rolling_ticker.index, short_rolling_ticker, label='20 days rolling')
        ax.plot(long_rolling_ticker.index, long_rolling_ticker, label='100 days rolling')

        ax.set_xlabel('Date')
        ax.set_ylabel('Adjusted closing price ($)')
        ax.legend()
    close.to_csv("CSV/close.csv", index=True)


In [ ]:
def close_print2(stock_symbol):
    
    df = data.DataReader(stock_symbol, 'yahoo', start_date, end_date)
    close = df['Close']
    all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')  # B stands for business day frequency
    close = close.reindex(all_weekdays)
    close = close.fillna(method='ffill')

    for i in stock_symbol:
        # Get the ticker timeseries. This now returns a Pandas Series object indexed by date.
        ticker = close.loc[:, i]

        # Calculate the 20 and 100 days moving averages of the closing prices
        short_rolling_ticker = ticker.rolling(window=20).mean()
        long_rolling_ticker = ticker.rolling(window=100).mean()

        # Plot everything by leveraging the very powerful matplotlib package
        fig, ax = plt.subplots(figsize=(18,11))

        plt.title(i)

        ax.plot(ticker.index, ticker, label=i)
        ax.plot(short_rolling_ticker.index, short_rolling_ticker, label='20 days rolling')
        ax.plot(long_rolling_ticker.index, long_rolling_ticker, label='100 days rolling')

        ax.set_xlabel('Date')
        ax.set_ylabel('Adjusted closing price ($)')
        ax.legend()
    close.to_csv("CSV/close.csv", index=True)


In [ ]:
close_print(['AAPL'])

In [ ]:
def volume_print(stock_symbol):
    
    df = data.DataReader(stock_symbol, 'yahoo', start_date, end_date)
    volume = df['Volume']
    all_weekdays = pd.date_range(start=start_date, end=end_date, freq='B')  # B stands for business day frequency
    volume = volume.reindex(all_weekdays)
    volume = volume.fillna(method='ffill')


    for i in stock_symbol:
        # Get the ticker timeseries. This now returns a Pandas Series object indexed by date.
        ticker = volume.loc[:, i]

        # Calculate the 30 and 100 days moving averages of the closing prices
        short_rolling_ticker = ticker.rolling(window=30).mean()
        long_rolling_ticker = ticker.rolling(window=100).mean()

        # short_rolling_ticker = ticker.rolling(window=30, center=True).mean()
        # long_rolling_ticker = ticker.rolling(window=100, center=True).mean()

        # Plot everything by leveraging the very powerful matplotlib package
        fig, ax = plt.subplots(figsize=(18,11))

        plt.title(i)

        ax.plot(ticker.index, ticker, label=i)
        ax.plot(short_rolling_ticker.index, short_rolling_ticker, label='30 days rolling')
        ax.plot(long_rolling_ticker.index, long_rolling_ticker, label='100 days rolling')

        ax.set_xlabel('Date')
        ax.set_ylabel('Stock Volume')
        ax.legend()

    volume.to_csv("CSV/volume.csv", index=True)


In [ ]:
volume_print(['AAPL'])

In [ ]:
# Relative returns
returns = close.pct_change(1)
returns.head()

In [ ]:
# Log returns - First the logarithm of the prices is taken and the the difference of consecutive (log) observations
log_returns = np.log(close).diff()
log_returns.head()

In [ ]:
# fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(18,13))

# for c in log_returns:
#     ax1.plot(log_returns.index, log_returns[c].cumsum(), label=str(c))

# ax1.set_ylabel('Cumulative log returns')
# ax1.legend(loc='best')

# for c in log_returns:
#     ax2.plot(log_returns.index, 100*(np.exp(log_returns[c].cumsum()) - 1), label=str(c))

# ax2.set_ylabel('Total relative returns (%)')
# ax2.legend(loc='best')

# plt.show()
